### Tauha Imran 22i1239 cs-g Ai Lab 6

# **Task 1 : Delivery Route Optimization with Beam Search**

A delivery company has multiple trucks, each capable of carrying a set number of packages. Your goal is to assign packages to trucks in a way that minimizes the total delivery time. Due to the large number of possible assignments, you will use Local Beam Search to efficiently explore the best delivery assignments.

### **Input:**

- A list of N packages, each with a specific delivery time (in minutes).
    - Example: packages = [20, 35, 10, 25, 40, 15, 30, 22]
- A set of M trucks.
    - Example: M = 3
- Beam width (number of best states to keep at each step).
    - Example: Beam Width = 3

### **Output:**
- A schedule showing which package is assigned to which truck.
- The total delivery time for each truck.

#### **Expected Output**
```
Best Package-Truck Assignment:
Truck 1: [20, 35]  --> Total Time: 55 minutes
Truck 2: [10, 40, 15]  --> Total Time: 65 minutes
Truck 3: [25, 30, 22]  --> Total Time: 77 minutes
```


### **Constraints:**
- The goal is to minimize the total delivery time (i.e., minimize the maximum load across all trucks).
- The heuristic function should evaluate states based on makespan ( **Makespan = max(Total time for each truck)** ).
- Keep only the k-best states at each step (k = beam width).


### **Algorithm Steps**

1. **Initialize Parameters**  
   - Read **N packages**, **M trucks**, and **Beam Width (k)**.  
   - Start with `k` initial states by assigning the first few packages in different ways.  

2. **Iterate Until All Packages Are Assigned**  
   - For each state in the beam, generate new states by assigning the next package to any truck.  
   - Compute **makespan** for each state:  
     ```
     Makespan = max(total delivery time of any truck)
     ```
   - Keep only the `k` best states (lowest makespan).  

3. **Terminate & Output Best Assignment**  
   - Once all packages are assigned, return the state with the **minimum makespan**.  
   - Output **truck assignments** and **total delivery time per truck**.  



In [6]:
# Adjust the code accordingly

def calculate_makespan(assignments):
    """
    Calculate the makespan, which is the maximum delivery time across all trucks.
    """
    #truck_times = [sum(assignments[truck]) for truck in assignments]
    truck_times = [sum(truck) for truck in assignments]
    return max(truck_times)
    #pass

def calculate__sub_makespan(truck):
  #truck_times = [sum(time) for time in truck]
  #return truck_times
  return sum(truck)


def get_successor_states(current_states, beam_width):
    """
    Generate successor states by slightly modifying the current best states.
    """

    successor_states = []
    for state in current_states:
        for i in range(len(state)):
            for j in range(len(state)):
                if i != j and state[i]:
                    #creating a sorta deep copy as new states...
                    new_state = [truck[:] for truck in state]
                    new_state[j].append(new_state[i].pop())
                    successor_states.append(new_state)

    #returns beam_width number of states sorted according to makespan
    return sorted(successor_states, key=calculate_makespan)[:beam_width]
    #pass


def beam_search(packages, num_trucks, beam_width, max_iterations=100):
    """
    Perform Beam Search to find an optimal package-truck assignment.
    """
    initial_state = [[] for _ in range(num_trucks)]
    for package in packages:
        initial_state[0].append(package)

    states = [initial_state]
    for _ in range(max_iterations):
        states = get_successor_states(states, beam_width)

    return min(states, key=calculate_makespan)
    #pass

def print_truck(optimal_assignment):
  for i in range(len(optimal_assignment)):
    print(f"Truck {i+1}: {optimal_assignment[i]}  --> Total Time: {calculate__sub_makespan(optimal_assignment[i])} minutes")


packages = [20, 35, 10, 25, 40, 15, 30, 22]  # Package delivery times
num_trucks = 3  # Number of trucks available
beam_width = 2  # Number of best states to keep

# Running Beam Search
optimal_assignment = beam_search(packages, num_trucks, beam_width)
print("Optimal Assignment:", optimal_assignment)
print("Makespan:", calculate_makespan(optimal_assignment))
print_truck(optimal_assignment)

Optimal Assignment: [[20, 35], [22, 30, 15], [40, 25, 10]]
Makespan: 75
Truck 1: [20, 35]  --> Total Time: 55 minutes
Truck 2: [22, 30, 15]  --> Total Time: 67 minutes
Truck 3: [40, 25, 10]  --> Total Time: 75 minutes


# **Task 2 : Finding the Best Seat in a Movie Theater Using Simulated Annealing**

You are in a crowded movie theater, trying to find the best seat. The goal is to get a seat that balances viewing experience (distance from the screen) and comfort (avoiding noisy neighbors). However, the best seats may not always be available, and you may need to explore different options before settling on one.


Each seat has a comfort score based on:

- Row Distance: How far the seat is from the middle row.
- Column Distance: How far the seat is from the middle column.
- Filled neighboring Seats: The number of occupied seats nearby.

Your goal is to find the best available seat using Simulated Annealing, optimizing for comfort while balancing exploration and exploitation.

### **Objective Function (Seat Score)**
Each seat’s discomfort is calculated as:
    
##### `𝐷 = Row Distance + Column Distance + Filled Neighbors seats`

A lower score means a better seat.

### **Temprature Decay after each iteration**

T=T-1




## **Algorithm Steps**
1. **Start at a seat (last row , last column).**
2. **Pick a valid (not occupied) neighboring seat randomly** (move up, down, left, or right).
3. **Compare discomfort scores**:
   - If the new seat is **better (lower D)** than the current one, move there.
   - If it’s **worse (higher D)**, accept it with probability:

     $$
     P = e^{-\frac{\Delta D}{T}}
     $$

     where:
     - \( ${\Delta D}$ = Distance of current seat - Distance of new seat \)
     - \( T \) is the current temperature.

   - If the probability \( P \) is **greater than 0.8**, accept the move ( update the current ); otherwise, skip this neighbor and select the next.
4. **Update the best seat found so far** (if the distance of the current is less than best seat found).
5. **Reduce the temperature** after each step.
6. **Stop when**:
   - **Temperature drops below 20.**


In [11]:
import random
import math

# ⬜ -> not allowed
# 💺 -> empty seats
# ❌ -> occupied seats
seats = [
    ["⬜", "⬜", "💺", "💺", "💺", "⬜", "⬜"],
    ["⬜", "💺", "💺", "❌", "❌", "💺", "⬜"],
    ["💺", "💺", "💺", "💺", "💺", "💺", "💺"],
    ["💺", "❌", "💺", "💺", "💺", "❌", "💺"],
    ["💺", "💺", "💺", "❌", "💺", "💺", "💺"]
]

# write the code here accordingly

def pick_seat_randon(curr_seat,curr_pos,seats):

    x = curr_pos[0]
    y = curr_pos[1]

    # Define possible moves (up, down, left, right)
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    valid_moves = []

    # Check each move
    for move in moves:
        new_x = x + move[0]
        new_y = y + move[1]
        #print(new_x , new_y)
        # Ensure the new position is within bounds
        if 0 <= new_x < len(seats) and 0 <= new_y < len(seats[0]):
            if seats[new_x][new_y] == "💺":
                valid_moves.append((new_x, new_y))
                #print('bingo')

    #print(valid_moves)

    # If there are valid moves, pick one randomly
    if valid_moves:
        new_pos = random.choice(valid_moves)
        return new_pos
    else:
        return None  # No valid move found

  #pass

def seat_score(curr_seat,curr_pos,seats):
  #Row Distance: How far the seat is from the middle row.
  rd = abs(len(seats)//2 - curr_pos[0])
  #Column Distance: How far the seat is from the middle column.
  cd = abs(len(seats[0])//2 - curr_pos[1])

  #Filled neighboring Seats: The number of occupied seats nearby.
  fns = 0
  moves = [(-1, 0), (1, 0), (0, -1), (0, 1),(-1,-1),(1,1),(-1,1),(1,-1)]
  for move in moves:
    new_x = curr_pos[0] + move[0]
    new_y = curr_pos[1] + move[1]
    if 0 <= new_x < len(seats) and 0 <= new_y < len(seats[0]):
      if seats[new_x][new_y] == "❌": # or seats[new_x][new_y] == "⬜":
        fns += 1
  D = rd + cd + fns
  return D

def simulated_annealing(initial_solution, initial_temperature, cooling_rate,seats, max_iterations):
  curr_pos = (len(seats)-1,len(seats[0])-1)
  curr_seat = seats[curr_pos[0]][curr_pos[1]]
  D = seat_score(curr_seat,curr_pos,seats)
  #print(curr_pos)
  #best_seat = curr_seat
  #best_pos = curr_pos
  T = initial_temperature

  while T > 20 and max_iterations > 0:
    #for _ in range(max_iterations):
    new_pos = pick_seat_randon(curr_seat,curr_pos,seats)
    T = T - cooling_rate
    max_iterations -= 1
    #print(T)
    #print(new_pos)
    if new_pos is not None:
      new_seat = seats[new_pos[0]][new_pos[1]]
      #curr_pos = new_pos
      #curr_seat = new_seat
      new_D = seat_score(curr_seat,curr_pos,seats)
      P = math.exp(-(new_D - D)/T)
      #print(P)

      if new_D < D:
        curr_pos = new_pos
        curr_seat = new_seat
        D = new_D
      else:
         if P > 0.8:
          curr_pos = new_pos
          curr_seat = new_seat
          D = new_D

      #print(curr_pos)
      #print(curr_seat)
  return curr_pos,curr_seat
  pass


# output best seat so far
initial_solution = seats[-1][-1]  # Define your initial solution
initial_temperature = 25
cooling_rate = 1
max_iterations = 100

#print(seats[-1][-1] == "💺")
init_pos = (len(seats)-1,len(seats[0])-1)
print("Initial Solution:", initial_solution , " " , init_pos)
#print(len(seats),len(seats[0]))
initial_temperature = 25
print("for Temperature: ", initial_temperature)
best_solution = simulated_annealing(initial_solution, initial_temperature, cooling_rate,seats, max_iterations)
print("Best Solution:", best_solution)

initial_temperature = 50
print("for Temperature: ", initial_temperature)
best_solution = simulated_annealing(initial_solution, initial_temperature, cooling_rate,seats, max_iterations)
print("Best Solution:", best_solution)


initial_temperature = 75
print("for Temperature: ", initial_temperature)
best_solution = simulated_annealing(initial_solution, initial_temperature, cooling_rate,seats, max_iterations)
print("Best Solution:", best_solution)


initial_temperature = 100
print("for Temperature: ", initial_temperature)
best_solution = simulated_annealing(initial_solution, initial_temperature, cooling_rate,seats, max_iterations)
print("Best Solution:", best_solution)

Initial Solution: 💺   (4, 6)
for Temperature:  25
Best Solution: ((2, 5), '💺')
for Temperature:  50
Best Solution: ((4, 4), '💺')
for Temperature:  75
Best Solution: ((3, 0), '💺')
for Temperature:  100
Best Solution: ((3, 3), '💺')
